<a href="https://colab.research.google.com/github/wonjae124/2021_Lab_project/blob/main/Compression_Autoencoder_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Set Dataset and model

## https://github.com/Po-Hsun-Su/pytorch-ssim -> 자료 참고해서 loss_function으로 SSIM 구하기


In [ ]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from torchvision import utils
from torchvision.utils import save_image
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio
from pytorch_msssim import ms_ssim  
import math
import time
from torchsummary import summary
import os
import matplotlib.pyplot as plt
from pytorch_msssim import ms_ssim
from pytorchtools import EarlyStopping
import pytorchtools

%matplotlib inline

ImportError: ignored

In [ ]:
from pytorchtools import EarlyStopping


ImportError: ignored

In [ ]:
import pytorchtools


In [ ]:
def save_checkpoint(epoch, model, optimizer, path): 
    state = { 'Epoch': epoch,
             'State_dict': model.state_dict(),
             'optimizer': optimizer.state_dict() 
             }
    torch.save(state, path)

In [ ]:
batch_size = 32 #minibatch가 2024일 때 GPU 사용량 6GB이므로 과하다.
torch.manual_seed(1)
torch.cuda.manual_seed(1)

##Original Train/Validation/Test dataset

In [ ]:
org_tr_set = datasets.STL10(root = "./data", split = "unlabeled", transform = transforms.ToTensor(), download=True)
tr_subset, val_subset = torch.utils.data.random_split(org_tr_set,[80000,20000],generator=torch.Generator().manual_seed(1))
test_set = datasets.STL10(root = "./data", split = "test",transform = transforms.ToTensor(), download=True)

In [ ]:
plt.imshow(org_tr_set[0][0].numpy().transpose(1,2,0))

In [ ]:
tr_dataloader = DataLoader(tr_subset, batch_size = batch_size, num_workers = 0) #pin_memory 삭제하고 num_workers=0으로 변경

val_dataloader = DataLoader(val_subset, batch_size = batch_size, num_workers = 0)

test_dataloader = DataLoader(test_set, batch_size = batch_size, num_workers = 0)

In [ ]:
print(len(tr_subset),len(val_subset), len(test_set))

# Model. Setting parameter -> float

In [ ]:
class Autoencoder(nn.Module): # 잠재벡터 크기 설정(하드 코딩)
  def __init__(self):
    super(Autoencoder, self).__init__()
    latent_dim = 256
    latent_img_size_reduce_2 = 3
    latent_img_size_reduce_8 = 9
    latent_img_size_reduce_16 = 17 

    self.encoder = nn.Sequential(
    # 96 by 96 by 3
        nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 2, stride = 2, padding=0),
        nn.BatchNorm2d(32),
    # 48 by 48 by 32
        nn.ReLU(True),


        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 2, stride = 2, padding = 0),
        nn.BatchNorm2d(64),
        nn.ReLU(True),
    # 24 by 24 by 64  

        nn.Conv2d(in_channels =64, out_channels = 128,  kernel_size = 2, stride = 2, padding=0),
    # 12 by 12 by 128 
        nn.BatchNorm2d(128),
        nn.ReLU(True),


        nn.Conv2d(in_channels = 128, out_channels = 256,  kernel_size = 2, stride = 2, padding=0),
        nn.BatchNorm2d(256),
    # 6 by 6 by 256
        nn.ReLU(True),

        nn.Conv2d(in_channels = 256, out_channels = latent_dim,  kernel_size = latent_img_size_reduce_2, stride = 1, padding=0), # latent_vector Z
        nn.BatchNorm2d(latent_dim),
        nn.ReLU(True),       
    )
    self.decoder = nn.Sequential(
    # 6 by 6 by 256

        nn.ConvTranspose2d(in_channels = latent_dim, out_channels = 256,  kernel_size = latent_img_size_reduce_2, stride = 1, padding=0),
        nn.BatchNorm2d(256),
        nn.ReLU(True),

        nn.ConvTranspose2d(in_channels = 256, out_channels = 128,  kernel_size = 2, stride = 2, padding=0),
        nn.BatchNorm2d(128),
        nn.ReLU(True),

    # 12 by 12 by 128  
        
        nn.ConvTranspose2d(in_channels = 128, out_channels = 64,  kernel_size = 2, stride = 2, padding=0),
        nn.BatchNorm2d(64),
        nn.ReLU(True),


    # 24 by 24 by 64
        nn.ConvTranspose2d(in_channels = 64, out_channels = 32,  kernel_size = 2, stride = 2, padding=0),
        nn.BatchNorm2d(32),
        nn.ReLU(True),   
    
    # 48 by 48 by 32

        nn.ConvTranspose2d(in_channels = 32, out_channels = 3, kernel_size = 2, stride = 2, padding=0),

    # 96 by 96 by 3
        nn.Sigmoid())
  
  def forward(self, x):
        latent_vector = self.encoder(x)
        decoded = self.decoder(latent_vector)
        return decoded

In [ ]:
n_epochs = 150
#시각화 횟수 설정 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

mse_loss_fn = torch.nn.MSELoss()
#ms_ssim_loss = ms_ssim()
#ms_ssim_val = ms_ssim( X, Y, data_range=1, size_average=False ) #(N,)


model = Autoencoder()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay  = 1e-04)
model=model.to(device)

train_loss_graph = np.array([]) # 손실 그래프 
val_loss_graph = np.array([])   

start = time.time()

#initial
early_stopping = EarlyStopping(patience = 20, verbose = True)


for epoch in range(1, n_epochs+1):
    print(f"Epoch : [{epoch}/{n_epochs}]","\n")

    model.train()
    train_loss = 0.0       #훈련 손실값
    total_train_loss = 0.0 
    val_loss = 0.0         #검증 손실값
    total_val_loss = 0.0

    #훈련
    for i,(tr_img, _) in enumerate(tr_dataloader):

        tr_img = tr_img.to(device)       #입력 이미지
        tr_outputs =model(tr_img) #복원 이미지
        optimizer.zero_grad()

        loss = mse_loss_fn(tr_outputs, tr_img).to(device)
        #loss = ms_ssim_loss(tr, Y, data_range=255, size_average=False )
        loss.backward()
        optimizer.step()

        train_loss = loss.clone().detach().item()
        total_train_loss += train_loss

    total_train_loss = total_train_loss/len(tr_dataloader) #평균 훈련 손실값
    train_loss_graph = np.append(train_loss_graph, total_train_loss)

    #검증
    model.eval()
    with torch.no_grad():   
        optimizer_strong.zero_grad()
        
        for i, (val_img, _) in enumerate(val_dataloader):
                val_img = val_img.to(device)
                val_outputs =model(val_img)
                val_loss = mse_loss_fn(val_outputs, val_img)
                val_loss = val_loss.item()
                total_val_loss += val_loss
        total_val_loss = total_val_loss/len(val_dataloader)
        val_loss_graph = np.append(val_loss_graph, total_val_loss)
        print('Training Loss: {:.6f}\t validation Loss {:.6f}'.format(total_train_loss, total_val_loss))
        
        ## save_checkpoint
        early_stopping(total_val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

            
        if (epoch)%5 == 0:

          for j in range(5):
            val_ssim = ssim(val_img[j].cpu().numpy().transpose(1, 2, 0), val_outputs[j].cpu().numpy().transpose(1, 2, 0), multichannel=True, gaussian_weights=True, use_sample_covariance=False)
    
            val_psnr = peak_signal_noise_ratio(val_img[j].cpu().numpy().transpose(1,2,0), val_outputs[j].cpu().numpy().transpose(1,2,0))
            print('PNSR :{:.2f} \t SSIM : {:.2f}'.format(val_psnr,val_ssim))
            
            ax = plt.subplot(2,5,j+1)
            plt.imshow(val_img[j].cpu().numpy().transpose(1, 2, 0))
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

            ax = plt.subplot(2,5,j+5+1)
            plt.imshow(val_outputs[j].cpu().numpy().transpose(1, 2, 0))
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

          plt.show()   

end = time.time()

total_time = (end-start)/60

model.load_state_dict(torch.load('checkpoint.pt'))

print("execution time : {:.1f}min".format(total_time))




In [ ]:
minposs = valid_loss.index(min(valid_loss))+1 
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')




fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(train_loss)+1),train_loss, label='Training Loss')
plt.plot(range(1,len(valid_loss)+1),valid_loss,label='Validation Loss'
plt.legend(['train_loss','val_loss'])
plt.title("Stl_10")
plt.xlabel("epochs")
plt.ylabel("loss")
plt.ylim(0, 0.5) # consistent scale
plt.xlim(0, len(train_loss)+1) # consistent scale
plt.tight_layout()
plt.show()

fig.savefig('loss_plot.png', bbox_inches='tight')


In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# file_path = '/content/drive/MyDrive/Colab Notebooks/kodag_5'
# names = ['04','10','17','18','19']

# model.eval()
# data = torchvision.datasets.ImageFolder( root = file_path, transform=transforms.ToTensor() )

# for  i in range(5):
#     kodag_img = data[i][0].unsqueeze(0).to(device)
#     kodag_output = model(kodag_img)
#     save_image(kodag_output,'reconstruction_kodag'+names[i]+'.png')
#     val_ssim = ssim(kodag_img.squeeze(0).detach().cpu().numpy().transpose(1, 2, 0), kodag_output.squeeze(0).detach().cpu().numpy().transpose(1, 2, 0), multichannel=True, gaussian_weights=True, use_sample_covariance=False)
#     val_psnr = Get_psnr(kodag_img, kodag_output)
#     print('PNSR :{:.2f} \t SSIM : {:.2f}'.format(val_psnr,val_ssim))


In [ ]:
# kodag_img.shape # 입력 이미지 차원 크기

In [ ]:
# kodag_output.shape # 출력 이미지 차원 크기

In [ ]:
# # print(kodag_img[0].shape)
# print(kodag_output[0].shape)

In [ ]:
# plt.imshow(kodag_output.detach().cpu().numpy().squeeze(0).transpose(1,2,0))
# save_image(kodag_output[0],'reconstruction_kodak19.png')

In [ ]:
# plt.imshow(kodag_img[0].detach().cpu().numpy().transpose(1,2,0))

In [ ]:
# plt.imshow(kodag_output[0].detach().cpu().numpy().transpose(1,2,0))

In [ ]:
# # 모델의 state_dict 출력
# print("Model's state_dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# # 옵티마이저의 state_dict 출력
# print("Optimizer's state_dict:")
# for var_name in optimizer_strong.state_dict():
#     print(var_name, "\t", optimizer_strong.state_dict()[var_name])

## test graph for (average psnr, ssim) 미완성

In [ ]:
# #TODO. 복원 이미지 그래프 그리기


# sum_ssim = 0.0
# sum_psnr = 0.0
# avg_ssim = 0.0
# avg_psnr = 0.0

# model.eval()
# with torch.no_grad():
#     optimizer_strong.zero_grad()
#     for i, (test_img, hat_test_img) in enumerate(zip(test_dataloader, Gs_blur_test_dataloader)):
        
#         test_img, _ = test_img
#         test_img = test_img.to(device)
#         test_hat_img, _ = hat_test_img
#         test_hat_img = test_hat_img.to(device)

#         test_outputs = model(test_hat_img)
        
#     for j in range(10):
#         img1_cpu = test_img[j].cpu().numpy().transpose(1, 2, 0)
#         img1_float64 = img1_cpu.astype(np.float64)
#         img2_cpu = test_outputs[j].cpu().numpy()
#         img2_float64 = img2_cpu.astype(np.float64).transpose(1, 2, 0)
        
#         ssim = ssim(img1_float64, img2_float64, multichannel=True, gaussian_weights=True)
#         psnr = peak_signal_noise_ratio(test_img[j], test_outputs[j], max_val = 1.0)
#         sum_psnr+=psnr
#         sum_ssim+=ssim
#         print('PNSR :{:.2f} \t SSIM : {:.2f}'.format(psnr,ssim))
        
#         ax = plt.subplot(3,10,j+1)
#         plt.imshow(test_img[j].cpu().numpy().transpose(1, 2, 0))
#         ax.get_xaxis().set_visible(False)
#         ax.get_yaxis().set_visible(False)


#         ax = plt.subplot(3,10,j+10+1)
#         plt.imshow(test_hat_img[j].cpu().numpy().transpose(1, 2, 0))
#         ax.get_xaxis().set_visible(False)
#         ax.get_yaxis().set_visible(False)


#         ax = plt.subplot(3,10, j+20+1)
#         plt.imshow(test_outputs[j].cpu().detach().numpy().transpose(1, 2, 0))
#         ax.get_xaxis().set_visible(False)
#         ax.get_yaxis().set_visible(False)

#     plt.show()   
# avg_psnr=sum_psnr/10
# avg_ssim=sum_ssim/10

# print("avg_psnr : {:.2f}".format(avg_psnr))
# print("avg_ssim : {:.2f}".format(avg_ssim))


In [ ]:
# !pip install pytorchtools

In [ ]:
#!pip install pytorch-msssim